<style>
.jp-Notebook {
    padding: var(--jp-notebook-padding);
    margin-left: 160px;
    outline: none;
    overflow: auto;
    background: var(--jp-layout-color0);
}
</style>

In [1]:
import pandas as pd
# Note you will likely have to change these paths. 
# If your data is in the same folder as this project, 
# the paths will likely be fixed for you by deleting ../../Data/awards_project/ from each string.
awards = pd.read_csv("awards_data.csv")
print('Successfully uploaded awards_data.csv.')

player_data = pd.read_csv("player_stats.csv")
print('Successfully uploaded player_stats.csv.')

team_data = pd.read_csv("team_stats.csv")
print('Successfully uploaded team_stats.csv. ')

rebounding_data = pd.read_csv("team_rebounding_data_22.csv")
print('Successfully uploaded team_rebounding_data_22.')


Successfully uploaded awards_data.csv.
Successfully uploaded player_stats.csv.
Successfully uploaded team_stats.csv. 
Successfully uploaded team_rebounding_data_22.


In [2]:
# Check if awards dataframe is spanning 2007-2021 data...
season_column = awards["season"]
min_season = season_column.min()
max_season = season_column.max()

print(f"Min season: {min_season}")
print(f"Max season: {max_season}")

Min season: 2007
Max season: 2021


Let's do a descriptive checking before doing any calculations.

In [3]:
#Step 1: extract 2010 drafted players, put them into a list.
drafted2010_list = list(set(player_data[player_data["draftyear"] == 2010]["nbapersonid"].tolist()))
#check if it's 73
if len(drafted2010_list) == 73:
    print("It's 73, let's move on!")
else:
    print("No. You got ", len(drafted2010_list))

player_dict = {nbapersonid: [] for nbapersonid in drafted2010_list}

It's 73, let's move on!


In [4]:
#Goal: proportionate the player_data according to game played...
player_data_modified = player_data.copy()

# Iterate through the player_data DataFrame and match players in 2010drafted_list
for index, row in player_data.iterrows():
    nbapersonid = row["nbapersonid"]
    if nbapersonid in drafted2010_list:
        season = row["season"]
        nbateamid = row["nbateamid"]
        
        # Find the corresponding row in team_data for the team's games count
        team_data_row = team_data[(team_data["season"] == season) & \
                                  (team_data["nbateamid"] == nbateamid)]
        
        if not team_data_row.empty:
            games_played = team_data_row["games"].iloc[0]
            if games_played != 82:
                x = row["games_start"] * (82 / games_played)
                y = row["mins"] * (82 / games_played)
                player_data_modified.at[index, "games_start"] = x
                player_data_modified.at[index, "mins"] = y


In [5]:
#Goal: I will make a dictionary
#key: nbapersonid , value: list(honors, honors, ...)

# Iterate through the drafted2010_list and check awards and player_data_modified DataFrames

def verdictSeason(season):
    for nbapersonid in drafted2010_list:
        #print('Now we dealing with', nbapersonid)
        elite_flag = False  # Flag to track if player is already categorized as 'Elite'
        all_star_flag = False  # Flag to track if player is already categorized as 'All-Star'
        
        for index, row in awards.iterrows():
            if row['nbapersonid'] == nbapersonid and row['season'] == season:
                # Check if the player won any All NBA award (1st, 2nd, or 3rd team), MVP, or DPOY
                if row['All NBA First Team'] == 1 or \
                row['All NBA Second Team'] == 1 or \
                row['All NBA Third Team'] == 1 or \
                row['Most Valuable Player_rk'] == 1 or \
                row['Defensive Player Of The Year_rk'] == 1:
                    player_dict[nbapersonid].append('Elite')
                    elite_flag = True  # Set flag to True

                # Check if the player was selected as an All-Star
                elif row['all_star_game'] == 1:
                    player_dict[nbapersonid].append('All-Star')
                    all_star_flag = True  # Set flag to True

        if not elite_flag and not all_star_flag:  # Skip lower-level verifications if already 'Elite' or 'All-Star'
            # Find the player's entry in player_data_modified
            player_entry = player_data_modified[(player_data_modified['nbapersonid'] == nbapersonid) & \
                                                (player_data_modified['season'] == season)]
            if not player_entry.empty:
                player_entry = player_entry.iloc[0]

                # Check if the player is a "Starter" or "Rotation" or "Roster"
                if player_entry['games_start'] >= 41 or player_entry['mins'] >= 2000:
                    player_dict[nbapersonid].append('Starter')
                elif player_entry['mins'] >= 1000:
                    player_dict[nbapersonid].append('Rotation')
                else:
                    player_dict[nbapersonid].append('Roster')
            else:
                # If player is not found in player_data, assume "Out of the League"
                player_dict[nbapersonid].append('Out of the League')

for season in range(2010,2022):
    #print('Now we are dealing with', season)
    verdictSeason(season)


# Print the updated player_dictionary
print(player_dict)


{202880: ['Out of the League', 'Roster', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League'], 202498: ['Out of the League', 'Roster', 'Roster', 'Roster', 'Rotation', 'Rotation', 'Roster', 'Rotation', 'Roster', 'Roster', 'Out of the League', 'Out of the League'], 202371: ['Roster', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League'], 202374: ['Roster', 'Roster', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League', 'Out of the League'], 1626246: ['Out of the League', 'Out of the League', 'Out of the League', 'Out of the Lea

In [6]:
#Let's take a look at my progress...
df_2010player = pd.DataFrame(player_dict.items(), columns=['nbapersonid', 'highest_acheived'])
df_2010player

,nbapersonid,highest_acheived
0,202880,"[Out of the League, Roster, Out of the League,..."
1,202498,"[Out of the League, Roster, Roster, Roster, Ro..."
2,202371,"[Roster, Out of the League, Out of the League,..."
3,202374,"[Roster, Roster, Out of the League, Out of the..."
4,1626246,"[Out of the League, Out of the League, Out of ..."
...,...,...
68,202362,"[Roster, Roster, Starter, Starter, Rotation, R..."
69,202363,"[Out of the League, Out of the League, Roster,..."
70,202620,"[Out of the League, Out of the League, Out of ..."
71,202365,"[Roster, Roster, Roster, Out of the League, Ou..."


In [7]:
#Dynamic Programming to calculate career outcome
def calculate_career_outcome(awardList):
    rankings = ['Elite', 'All-Star', 'Starter', 'Rotation', 'Roster', 'Out of the League']
    years_count = [0] * 6
    for i in range(4,len(awardList)):
        rank = rankings.index(awardList[i])
        for t in range (rank,len(years_count)):
            years_count[t] += 1
    #print(years_count)
    for p in range(6):
        if years_count[p] >= 2:
            return rankings[p]
    
        
    

# Example awards list from 2010 to 2021
df_2010player['career_outcome'] = df_2010player['highest_acheived'].apply(calculate_career_outcome)

print(df_2010player[:10])


   nbapersonid                                   highest_acheived  \
0       202880  [Out of the League, Roster, Out of the League,...   
1       202498  [Out of the League, Roster, Roster, Roster, Ro...   
2       202371  [Roster, Out of the League, Out of the League,...   
3       202374  [Roster, Roster, Out of the League, Out of the...   
4      1626246  [Out of the League, Out of the League, Out of ...   
5       202376  [Roster, Roster, Out of the League, Out of the...   
6       202377  [Roster, Out of the League, Out of the League,...   
7       202378  [Roster, Out of the League, Out of the League,...   
8       202379  [Roster, Roster, Roster, Rotation, Roster, Ros...   
9       202380  [Roster, Roster, Out of the League, Roster, Ou...   

      career_outcome  
0  Out of the League  
1           Rotation  
2  Out of the League  
3  Out of the League  
4             Roster  
5  Out of the League  
6  Out of the League  
7  Out of the League  
8             Roster  
9  Out of 

In [8]:
#Count them!
award_counts = df_2010player['career_outcome'].value_counts()
award_counts

Out of the League    38
Roster               14
Starter              11
Rotation              7
Elite                 2
All-Star              1
Name: career_outcome, dtype: int64

Let me check my answer based on my NBA knowledge.

There are four all-stars in 2010 draft class.
Cousins, Hayward, PG13, John Wall.
PG13 and Cousins are two elites. 
Wall only manage to get in 3rd team one time in 2017. So he is the All-Star.
Hayward, incredible player, suffers injuries. Only gets in All-Star in 2017. He doesn't count.


In [9]:
# Data integration - forming dataset:
# Note: we are only considering 2011-2015 drafted players as training set.
#Because 1. data integrity for the rookie data; 
#2. my personal mac does not have a strong chip to pursue large inputs.
import pandas as pd

training_data = pd.DataFrame()
person_col = []

for season in range(2007,2016):
    person_col += list(set(player_data_modified[player_data_modified["draftyear"] == season]["nbapersonid"].tolist()))

    
training_data['nbapersonid'] = person_col

#let's extract some useful info from player_data
for index, row in training_data.iterrows():
    nbapersonid = row['nbapersonid']
    player_row = player_data_modified[player_data_modified['nbapersonid'] == nbapersonid]
    
    #extractions
    draftyear = player_row['draftyear'].values[0]  #draft year
    draftpick = player_row['draftpick'].values[0]  #draft pick
    
    #fill-ups
    training_data.at[index, 'draftyear'] = draftyear
    training_data.at[index, 'draftpick'] = draftpick
    
training_data['draftpick'].fillna(61, inplace=True)
#To deal with undrafted players, let them be 61st drafted.
training_data

,nbapersonid,draftyear,draftpick
0,201858,2007.0,61.0
1,203141,2007.0,61.0
2,201229,2007.0,61.0
3,201234,2007.0,61.0
4,201235,2007.0,61.0
...,...,...,...
669,1626214,2015.0,61.0
670,1630698,2015.0,61.0
671,1626220,2015.0,61.0
672,1626224,2015.0,31.0


In [10]:
training_data['first_year_ROM_amt'] = 0
training_data['All_Rookie_First'] = 0
training_data['All_Rookie_Second'] = 0

# Iterate over each row in 'award' DataFrame
# Get the rookie year perfomance info from awards
for index, row in awards.iterrows():
    nbapersonid = row['nbapersonid']
    season = row['season']
    
    # Find the corresponding player's draft year in 'training_data'
    player_row = training_data[training_data['nbapersonid'] == nbapersonid]
    if player_row.empty:
        continue
    draftyear = player_row['draftyear'].values[0]

    first_year = draftyear
    last_year = draftyear + 3
    
    # Check if the award season falls within the player's first four years
    if first_year == season:
        training_data.at[player_row.index, 'first_year_ROM_amt'] = row['Rookie Of The Month']
        training_data.at[player_row.index, 'All_Rookie_First'] = row['All Rookie First Team']
        training_data.at[player_row.index, 'All_Rookie_Second'] = row['All Rookie Second Team'] 
        
        
player_awards = {}
for index, row in awards.iterrows():
    nbapersonid = row['nbapersonid']
    season = row['season']
    pow_award = row['Player Of The Week']
    
    # Find the corresponding player's draft year in 'training_data'
    player_row = training_data[training_data['nbapersonid'] == nbapersonid]
    if player_row.empty:
        continue
    
    draftyear = player_row['draftyear'].values[0]
    
    # Calculate the first year of the player
    first_year = draftyear
    last_year = draftyear + 3
    
    # Check if the award season falls within the player's first four years
    if first_year <= season <= last_year:
        if nbapersonid not in player_awards:
            player_awards[nbapersonid] = 0
        player_awards[nbapersonid] += pow_award

# Calculate and assign 'four_yrs_POW' to 'training_data' DataFrame
training_data['four_yrs_POW'] = training_data['nbapersonid'].map(player_awards)

In [11]:
training_data.fillna(0, inplace=True)

In [12]:
rookie_stats = {}

# Iterate over each row in 'player_data' DataFrame
for index, row in player_data_modified.iterrows():
    nbapersonid = row['nbapersonid']
    season = row['season']
    
    # Find the corresponding player's draft year in 'training_data'
    player_row = training_data[training_data['nbapersonid'] == nbapersonid]
    if player_row.empty:
        continue
    
    draftyear = player_row['draftyear'].values[0]
    
    # Calculate the first year of the player
    first_year = draftyear
    last_year = draftyear + 3
    
    # Check if the player's rookie season matches the first year in 'player_data'
    if first_year == season:
        rookie_stats[nbapersonid] = {
            'Rookie_ppg': row['points']/row['games'],
            'Rookie_apg': row['ast']/row['games'],
            'Rookie_rpg': row['tot_reb_pct']/row['games'],
            'Rookie_tov_pct': row['tov_pct'],
            'Rookie_mins': row['mins']
        }

# Update the 'training_data' DataFrame with rookie statistics
training_data['Rookie_ppg'] = training_data['nbapersonid'].map \
(lambda x: rookie_stats[x]['Rookie_ppg'] if x in rookie_stats else 0)
training_data['Rookie_apg'] = training_data['nbapersonid'].map \
(lambda x: rookie_stats[x]['Rookie_apg'] if x in rookie_stats else 0)
training_data['Rookie_rpg'] = training_data['nbapersonid'].map \
(lambda x: rookie_stats[x]['Rookie_rpg'] if x in rookie_stats else 0)
training_data['Rookie_tov_pct'] = training_data['nbapersonid'].map \
(lambda x: rookie_stats[x]['Rookie_tov_pct'] if x in rookie_stats else 0)
training_data['Rookie_mins'] = training_data['nbapersonid'].map \
(lambda x: rookie_stats[x]['Rookie_mins'] if x in rookie_stats else 0)


In [13]:
player_data_modified['ppg'] = player_data_modified['points']/player_data_modified['games']
player_data_modified['rpg'] = player_data_modified['tot_reb']/player_data_modified['games']
player_data_modified['apg'] = player_data_modified['ast']/player_data_modified['games']

In [14]:
first_four_ppg = {}
first_four_apg = {}
first_four_rpg = {}
first_four_mins = {}
first_four_games = {}
first_four_games_start = {}

# Iterate over each row in 'training_data' DataFrame
for index, row in training_data.iterrows():
    nbapersonid = row['nbapersonid']
    draftyear = row['draftyear']
    
    # Calculate the first year and last year for the player
    first_year = draftyear
    last_year = first_year + 3
    
    # Calculate the average ppg for the first four years
    avg_ppg = player_data_modified[
        (player_data_modified['nbapersonid'] == nbapersonid) & 
        (player_data_modified['season'] >= first_year) & 
        (player_data_modified['season'] <= last_year)
    ]['ppg'].mean()
    
    avg_apg = player_data_modified[
        (player_data_modified['nbapersonid'] == nbapersonid) & 
        (player_data_modified['season'] >= first_year) & 
        (player_data_modified['season'] <= last_year)
    ]['apg'].mean()
    
    avg_rpg = player_data_modified[
        (player_data_modified['nbapersonid'] == nbapersonid) & 
        (player_data_modified['season'] >= first_year) & 
        (player_data_modified['season'] <= last_year)
    ]['rpg'].mean()
    
    tot_games = player_data_modified[
        (player_data_modified['nbapersonid'] == nbapersonid) & 
        (player_data_modified['season'] >= first_year) & 
        (player_data_modified['season'] <= last_year)
    ]['games'].sum()

    tot_mins = player_data_modified[
        (player_data_modified['nbapersonid'] == nbapersonid) & 
        (player_data_modified['season'] >= first_year) & 
        (player_data_modified['season'] <= last_year)
    ]['mins'].sum()

    tot_gs = player_data_modified[
        (player_data_modified['nbapersonid'] == nbapersonid) & 
        (player_data_modified['season'] >= first_year) & 
        (player_data_modified['season'] <= last_year)
    ]['games_start'].sum()
    
    first_four_ppg[nbapersonid] = avg_ppg
    first_four_apg[nbapersonid] = avg_apg
    first_four_rpg[nbapersonid] = avg_rpg
    first_four_mins[nbapersonid] = tot_mins
    first_four_games[nbapersonid] = tot_games
    first_four_games_start[nbapersonid] = tot_gs

# Calculate 'avg_4yr_ppg' for each nbapersonid in 'training_data'
training_data['avg_4yr_ppg'] = training_data['nbapersonid'].map(lambda x: first_four_ppg.get(x, 0))
training_data['avg_4yr_apg'] = training_data['nbapersonid'].map(lambda x: first_four_apg.get(x, 0))
training_data['avg_4yr_rpg'] = training_data['nbapersonid'].map(lambda x: first_four_rpg.get(x, 0))
training_data['tot_mins'] = training_data['nbapersonid'].map(lambda x: first_four_mins.get(x, 0))
training_data['tot_games'] = training_data['nbapersonid'].map(lambda x: first_four_games.get(x, 0))
training_data['tot_games_start'] = training_data['nbapersonid'].map(lambda x: first_four_games_start.get(x, 0))


In [15]:
training_data.fillna(0, inplace=True)

In [16]:
drafted2007_list = list(set(player_data_modified[player_data_modified["draftyear"] == 2007]["nbapersonid"].tolist()))
drafted2008_list = list(set(player_data_modified[player_data_modified["draftyear"] == 2008]["nbapersonid"].tolist()))
drafted2009_list = list(set(player_data_modified[player_data_modified["draftyear"] == 2009]["nbapersonid"].tolist()))
drafted2010_list = list(set(player_data_modified[player_data_modified["draftyear"] == 2010]["nbapersonid"].tolist()))
drafted2011_list = list(set(player_data_modified[player_data_modified["draftyear"] == 2011]["nbapersonid"].tolist()))
drafted2012_list = list(set(player_data_modified[player_data_modified["draftyear"] == 2012]["nbapersonid"].tolist()))
drafted2013_list = list(set(player_data_modified[player_data_modified["draftyear"] == 2013]["nbapersonid"].tolist()))
drafted2014_list = list(set(player_data_modified[player_data_modified["draftyear"] == 2014]["nbapersonid"].tolist()))
drafted2015_list = list(set(player_data_modified[player_data_modified["draftyear"] == 2015]["nbapersonid"].tolist()))
player_dict2007 = {nbapersonid: [] for nbapersonid in drafted2007_list}
player_dict2008 = {nbapersonid: [] for nbapersonid in drafted2008_list}
player_dict2009 = {nbapersonid: [] for nbapersonid in drafted2009_list}
player_dict2010 = {nbapersonid: [] for nbapersonid in drafted2010_list}
player_dict2011 = {nbapersonid: [] for nbapersonid in drafted2011_list}
player_dict2012 = {nbapersonid: [] for nbapersonid in drafted2012_list}
player_dict2013 = {nbapersonid: [] for nbapersonid in drafted2013_list}
player_dict2014 = {nbapersonid: [] for nbapersonid in drafted2014_list}
player_dict2015 = {nbapersonid: [] for nbapersonid in drafted2015_list}

def verdictSeason(season, players,player_dict):
    for nbapersonid in players:
        #print('Now we dealing with', nbapersonid)
        elite_flag = False  # Flag to track if player is already categorized as 'Elite'
        all_star_flag = False  # Flag to track if player is already categorized as 'All-Star'
        
        for index, row in awards.iterrows():
            if row['nbapersonid'] == nbapersonid and row['season'] == season:
                # Check if the player won any All NBA award (1st, 2nd, or 3rd team), MVP, or DPOY
                if row['All NBA First Team'] == 1 or \
                row['All NBA Second Team'] == 1 or \
                row['All NBA Third Team'] == 1 or \
                row['Most Valuable Player_rk'] == 1 or \
                row['Defensive Player Of The Year_rk'] == 1:
                    player_dict[nbapersonid].append('Elite')
                    elite_flag = True  # Set flag to True

                # Check if the player was selected as an All-Star
                elif row['all_star_game'] == 1:
                    player_dict[nbapersonid].append('All-Star')
                    all_star_flag = True  # Set flag to True

        if not elite_flag and not all_star_flag:  # Skip lower-level verifications if already 'Elite' or 'All-Star'
            # Find the player's entry in player_data_modified
            player_entry = player_data_modified[(player_data_modified['nbapersonid'] == nbapersonid) & \
                                                (player_data_modified['season'] == season)]
            if not player_entry.empty:
                player_entry = player_entry.iloc[0]

                # Check if the player is a "Starter" or "Rotation" or "Roster"
                if player_entry['games_start'] >= 41 or player_entry['mins'] >= 2000:
                    player_dict[nbapersonid].append('Starter')
                elif player_entry['mins'] >= 1000:
                    player_dict[nbapersonid].append('Rotation')
                else:
                    player_dict[nbapersonid].append('Roster')
            else:
                # If player is not found in player_data, assume "Out of the League"
                player_dict[nbapersonid].append('Out of the League')


for season in range(2007,2022):
    verdictSeason(season, drafted2007_list,player_dict2007)
for season in range(2008,2022):
    verdictSeason(season, drafted2008_list,player_dict2008)
for season in range(2009,2022):
    verdictSeason(season, drafted2009_list,player_dict2009)
for season in range(2010,2022):
    verdictSeason(season, drafted2010_list,player_dict2010)
for season in range(2011,2022):
    verdictSeason(season, drafted2011_list,player_dict2011)
for season in range(2012,2022):
    verdictSeason(season, drafted2012_list,player_dict2012)
for season in range(2013,2022):
    verdictSeason(season, drafted2013_list,player_dict2013)
for season in range(2014,2022):
    verdictSeason(season, drafted2014_list,player_dict2014)
for season in range(2015,2022):
    verdictSeason(season, drafted2015_list,player_dict2015)

combined_dict = {
    **player_dict2007,
    **player_dict2008,
    **player_dict2009,
    **player_dict2010,
    **player_dict2011,
    **player_dict2012,
    **player_dict2013,
    **player_dict2014,
    **player_dict2015
}

df_players = pd.DataFrame(combined_dict.items(), columns=['nbapersonid', 'highest_acheived'])
    
df_players['career_outcome'] = df_players['highest_acheived'].apply(calculate_career_outcome)

In [17]:
df_players['first_year_achieved'] = df_players['highest_acheived'].apply \
(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
df_players['second_year_achieved'] = df_players['highest_acheived'].apply \
(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
df_players['third_year_achieved'] = df_players['highest_acheived'].apply \
(lambda x: x[2] if isinstance(x, list) and len(x) > 2 else None)
df_players['fourth_year_achieved'] = df_players['highest_acheived'].apply \
(lambda x: x[3] if isinstance(x, list) and len(x) > 3 else None)

In [18]:
merged_data = pd.merge(training_data, df_players, on='nbapersonid', how='left')
print(training_data)
# Append the desired columns to the 'training_data' DataFrame
training_data['first_year_achieved'] = merged_data['first_year_achieved']
training_data['second_year_achieved'] = merged_data['second_year_achieved']
training_data['third_year_achieved'] = merged_data['third_year_achieved']
training_data['fourth_year_achieved'] = merged_data['fourth_year_achieved']
training_data['career_outcome'] = merged_data['career_outcome']



     nbapersonid  draftyear  draftpick  first_year_ROM_amt  All_Rookie_First  \
0         201858     2007.0       61.0                 0.0               0.0   
1         203141     2007.0       61.0                 0.0               0.0   
2         201229     2007.0       61.0                 0.0               0.0   
3         201234     2007.0       61.0                 0.0               0.0   
4         201235     2007.0       61.0                 0.0               0.0   
..           ...        ...        ...                 ...               ...   
669      1626214     2015.0       61.0                 0.0               0.0   
670      1630698     2015.0       61.0                 0.0               0.0   
671      1626220     2015.0       61.0                 0.0               0.0   
672      1626224     2015.0       31.0                 0.0               0.0   
673      1629308     2015.0       61.0                 0.0               0.0   

     All_Rookie_Second  four_yrs_POW  R

In [19]:
#Encode the awards.
mapping = {
    'Elite': 6,
    'All-Star': 5,
    'Starter': 4,
    'Rotation': 3,
    'Roster': 2,
    'Out of the League': 1
}

# Encode the specified columns
columns_to_encode = ['first_year_achieved', 'second_year_achieved', \
                     'third_year_achieved', 'fourth_year_achieved', 'career_outcome']
training_data[columns_to_encode] = training_data[columns_to_encode].replace(mapping)



In [20]:
training_data['simple_label'] = training_data['career_outcome'].apply(lambda x: 1 if x == 1 else (2 if x in [2, 3, 4] else 3))
# Display the updated 'training_data' DataFrame
training_data.to_csv('Training.csv')
training_data

,nbapersonid,draftyear,draftpick,first_year_ROM_amt,All_Rookie_First,All_Rookie_Second,four_yrs_POW,Rookie_ppg,Rookie_apg,Rookie_rpg,...,avg_4yr_rpg,tot_mins,tot_games,tot_games_start,first_year_achieved,second_year_achieved,third_year_achieved,fourth_year_achieved,career_outcome,simple_label
0,201858,2007.0,61.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,2.437019,1198,103,4,1,2,2,2,2,2
1,203141,2007.0,61.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0,0,0,1,1,1,1,3,2
2,201229,2007.0,61.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,3.611589,2997,130,33,1,2,3,3,3,2
3,201234,2007.0,61.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.416667,108,12,0,1,2,1,2,1,1
4,201235,2007.0,61.0,0.0,0.0,0.0,0.0,0.000000,0.333333,0.033000,...,1.555556,24,5,0,2,2,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,1626214,2015.0,61.0,0.0,0.0,0.0,0.0,5.642857,1.071429,0.006786,...,3.428571,279,14,11,2,1,1,1,1,1
670,1630698,2015.0,61.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0,0,0,1,1,1,1,1,1
671,1626220,2015.0,61.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,3.447950,2821,151,20,1,1,3,3,4,2
672,1626224,2015.0,31.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,3.332291,3116,137,87,1,1,2,4,3,2


In [21]:
testing_data = pd.DataFrame()
person_col = []


person_col += list(set(player_data_modified[player_data_modified["draftyear"] == 2018]["nbapersonid"].tolist()))

    
testing_data['nbapersonid'] = person_col

#let's extract some useful info from player_data
for index, row in testing_data.iterrows():
    nbapersonid = row['nbapersonid']
    player_row = player_data_modified[player_data_modified['nbapersonid'] == nbapersonid]
    
    #extractions
    draftyear = player_row['draftyear'].values[0]  #draft year
    draftpick = player_row['draftpick'].values[0]  #draft pick
    
    #fill-ups
    testing_data.at[index, 'draftyear'] = draftyear
    testing_data.at[index, 'draftpick'] = draftpick
    
testing_data['draftpick'].fillna(61, inplace=True)

testing_data['first_year_ROM_amt'] = 0
testing_data['All_Rookie_First'] = 0
testing_data['All_Rookie_Second'] = 0

# Iterate over each row in 'award' DataFrame
# Get the rookie year perfomance info from awards
for index, row in awards.iterrows():
    nbapersonid = row['nbapersonid']
    season = row['season']
    
    # Find the corresponding player's draft year in 'training_data'
    player_row = testing_data[testing_data['nbapersonid'] == nbapersonid]
    if player_row.empty:
        continue
    draftyear = player_row['draftyear'].values[0]

    first_year = draftyear
    last_year = draftyear + 3
    
    # Check if the award season falls within the player's first four years
    if first_year == season:
        testing_data.at[player_row.index, 'first_year_ROM_amt'] = row['Rookie Of The Month']
        testing_data.at[player_row.index, 'All_Rookie_First'] = row['All Rookie First Team']
        testing_data.at[player_row.index, 'All_Rookie_Second'] = row['All Rookie Second Team'] 
        
        
player_awards = {}
for index, row in awards.iterrows():
    nbapersonid = row['nbapersonid']
    season = row['season']
    pow_award = row['Player Of The Week']
    
    # Find the corresponding player's draft year in 'training_data'
    player_row = testing_data[testing_data['nbapersonid'] == nbapersonid]
    if player_row.empty:
        continue
    
    draftyear = player_row['draftyear'].values[0]
    
    # Calculate the first year of the player
    first_year = draftyear
    last_year = draftyear + 3
    
    # Check if the award season falls within the player's first four years
    if first_year <= season <= last_year:
        if nbapersonid not in player_awards:
            player_awards[nbapersonid] = 0
        player_awards[nbapersonid] += pow_award

# Calculate and assign 'four_yrs_POW' to 'training_data' DataFrame
testing_data['four_yrs_POW'] = testing_data['nbapersonid'].map(player_awards)

testing_data.fillna(0, inplace=True)

rookie_stats = {}

# Iterate over each row in 'player_data' DataFrame
for index, row in player_data_modified.iterrows():
    nbapersonid = row['nbapersonid']
    season = row['season']
    
    # Find the corresponding player's draft year in 'training_data'
    player_row = testing_data[testing_data['nbapersonid'] == nbapersonid]
    if player_row.empty:
        continue
    
    draftyear = player_row['draftyear'].values[0]
    
    # Calculate the first year of the player
    first_year = draftyear
    last_year = draftyear + 3
    
    # Check if the player's rookie season matches the first year in 'player_data'
    if first_year == season:
        rookie_stats[nbapersonid] = {
            'Rookie_ppg': row['ppg'],
            'Rookie_apg': row['ast']/row['games'],
            'Rookie_rpg': row['tot_reb_pct']/row['games'],
            'Rookie_tov_pct': row['tov_pct'],
            'Rookie_mins': row['mins']
        }

# Update the 'training_data' DataFrame with rookie statistics
testing_data['Rookie_ppg'] = testing_data['nbapersonid'].map \
(lambda x: rookie_stats[x]['Rookie_ppg'] if x in rookie_stats else 0)
testing_data['Rookie_apg'] = testing_data['nbapersonid'].map \
(lambda x: rookie_stats[x]['Rookie_apg'] if x in rookie_stats else 0)
testing_data['Rookie_rpg'] = testing_data['nbapersonid'].map \
(lambda x: rookie_stats[x]['Rookie_rpg'] if x in rookie_stats else 0)
testing_data['Rookie_tov_pct'] = testing_data['nbapersonid'].map \
(lambda x: rookie_stats[x]['Rookie_tov_pct'] if x in rookie_stats else 0)
testing_data['Rookie_mins'] = testing_data['nbapersonid'].map \
(lambda x: rookie_stats[x]['Rookie_mins'] if x in rookie_stats else 0)

first_four_ppg = {}
first_four_apg = {}
first_four_rpg = {}
first_four_mins = {}
first_four_games = {}
first_four_games_start = {}

# Iterate over each row in 'training_data' DataFrame
for index, row in training_data.iterrows():
    nbapersonid = row['nbapersonid']
    draftyear = row['draftyear']
    
    # Calculate the first year and last year for the player
    first_year = draftyear
    last_year = first_year + 3
    
    # Calculate the average ppg for the first four years
    avg_ppg = player_data_modified[
        (player_data_modified['nbapersonid'] == nbapersonid) & 
        (player_data_modified['season'] >= first_year) & 
        (player_data_modified['season'] <= last_year)
    ]['ppg'].mean()
    
    avg_apg = player_data_modified[
        (player_data_modified['nbapersonid'] == nbapersonid) & 
        (player_data_modified['season'] >= first_year) & 
        (player_data_modified['season'] <= last_year)
    ]['apg'].mean()
    
    avg_rpg = player_data_modified[
        (player_data_modified['nbapersonid'] == nbapersonid) & 
        (player_data_modified['season'] >= first_year) & 
        (player_data_modified['season'] <= last_year)
    ]['rpg'].mean()
    
    tot_games = player_data_modified[
        (player_data_modified['nbapersonid'] == nbapersonid) & 
        (player_data_modified['season'] >= first_year) & 
        (player_data_modified['season'] <= last_year)
    ]['games'].sum()

    tot_mins = player_data_modified[
        (player_data_modified['nbapersonid'] == nbapersonid) & 
        (player_data_modified['season'] >= first_year) & 
        (player_data_modified['season'] <= last_year)
    ]['mins'].sum()

    tot_gs = player_data_modified[
        (player_data_modified['nbapersonid'] == nbapersonid) & 
        (player_data_modified['season'] >= first_year) & 
        (player_data_modified['season'] <= last_year)
    ]['games_start'].sum()
    
    first_four_ppg[nbapersonid] = avg_ppg
    first_four_apg[nbapersonid] = avg_apg
    first_four_rpg[nbapersonid] = avg_rpg
    first_four_mins[nbapersonid] = tot_mins
    first_four_games[nbapersonid] = tot_games
    first_four_games_start[nbapersonid] = tot_gs

# Calculate 'avg_4yr_ppg' for each nbapersonid in 'training_data'
testing_data['avg_4yr_ppg'] = testing_data['nbapersonid'].map(lambda x: first_four_ppg.get(x, 0))
testing_data['avg_4yr_apg'] = testing_data['nbapersonid'].map(lambda x: first_four_apg.get(x, 0))
testing_data['avg_4yr_rpg'] = testing_data['nbapersonid'].map(lambda x: first_four_rpg.get(x, 0))
testing_data['tot_mins'] = testing_data['nbapersonid'].map(lambda x: first_four_mins.get(x, 0))
testing_data['tot_games'] = testing_data['nbapersonid'].map(lambda x: first_four_games.get(x, 0))
testing_data['tot_games_start'] = testing_data['nbapersonid'].map(lambda x: first_four_games_start.get(x, 0))
testing_data

,nbapersonid,draftyear,draftpick,first_year_ROM_amt,All_Rookie_First,All_Rookie_Second,four_yrs_POW,Rookie_ppg,Rookie_apg,Rookie_rpg,Rookie_tov_pct,Rookie_mins,avg_4yr_ppg,avg_4yr_apg,avg_4yr_rpg,tot_mins,tot_games,tot_games_start
0,1629185,2018.0,61.0,0.0,0.0,0.0,0.0,0.857143,0.571429,0.009571,0.111,33,0,0,0,0,0,0
1,1629203,2018.0,61.0,0.0,0.0,0.0,0.0,4.500000,2.166667,0.029500,0.129,143,0,0,0,0,0,0
2,1629216,2018.0,61.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000,0,0,0,0,0,0,0
3,1630256,2018.0,61.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000,0,0,0,0,0,0,0
4,1629234,2018.0,61.0,0.0,0.0,0.0,0.0,1.782609,0.304348,0.007130,0.201,113,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1629150,2018.0,61.0,0.0,0.0,0.0,0.0,4.500000,0.500000,0.083000,0.225,20,0,0,0,0,0,0
96,1629155,2018.0,61.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.500,4,0,0,0,0,0,0
97,1629162,2018.0,61.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000,0,0,0,0,0,0,0
98,1629164,2018.0,61.0,0.0,0.0,0.0,0.0,1.437500,0.875000,0.001750,0.097,57,0,0,0,0,0,0


In [28]:
pred_df = pd.read_csv('predicting.csv')
player_data_unique = player_data.drop_duplicates(subset='nbapersonid', keep='first')
merged_df = pred_df.merge(player_data_unique[['nbapersonid', 'player']], on='nbapersonid', how='left')
print(merged_df)

    Unnamed: 0  nbapersonid  draftyear  draftpick  first_year_ROM_amt  \
0            0      1629185     2018.0       61.0                 0.0   
1            1      1629203     2018.0       61.0                 0.0   
2            2      1629216     2018.0       61.0                 0.0   
3            3      1630256     2018.0       61.0                 0.0   
4            4      1629234     2018.0       61.0                 0.0   
..         ...          ...        ...        ...                 ...   
95          95      1629150     2018.0       61.0                 0.0   
96          96      1629155     2018.0       61.0                 0.0   
97          97      1629162     2018.0       61.0                 0.0   
98          98      1629164     2018.0       61.0                 0.0   
99          99      1629168     2018.0       61.0                 0.0   

    All_Rookie_First  All_Rookie_Second  four_yrs_POW  Rookie_ppg  Rookie_apg  \
0                0.0                0.0   

In [29]:
merged_df.to_csv('predicting_with_names.csv')